<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/11-Flask/I-All_Elements_Together.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# All pieces together

This is just a notebook that puts together all the pieces of the code that we have been writing during this tutorial.

In [ ]:
!pip install -U PyMySQL sqlalchemy flask pyngrok geopandas pygeos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import threading

from flask import Flask, render_template, jsonify, request
from pyngrok import ngrok

from sqlalchemy import create_engine
import pandas as pd
import base64
from io import BytesIO
import matplotlib.pyplot as plt

# Setup Flask and ngrok

os.environ["FLASK_DEBUG"] = "true"

app = Flask(__name__, template_folder = '/content/drive/MyDrive/templates')
port = 5000

# Open a ngrok tunnel to the HTTP server
ngrok_authtoken = '2EYf3qVk9mi739HjPwSNZXWAtfy_4jF9NAhGqVVVJmm4YehPW'
ngrok.set_auth_token(ngrok_authtoken)
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel '{public_url}' -> 'http://127.0.0.1:{port}'")

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# Setup a connection to the database
conn_string = 'mysql+pymysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    db = 'citibike_fall2017',
    password = 'dwdstudent2015',
    encoding = 'utf8mb4')
engine = create_engine(conn_string)

In [ ]:
# Our website has just two pages.


# The main page shows just a big box with a search interface
# The HTML for the page is at 
# https://github.com/ipeirotis/dealing_with_data/blob/master/11-Flask/search_stations.html
@app.route('/')
def home():
    return render_template('search_stations.html')

# This page shows the list of the stations
# Optionally, we also pass a query to the page with 
# the name of the station and we do an approximate search
#
# The HTML for the page is at 
# https://github.com/ipeirotis/dealing_with_data/blob/master/11-Flask/list_stations.html
@app.route('/list_stations',  methods=['GET'])
def list_stations():
  return render_template("list_stations.html")    

In [ ]:
# This is the API call that returns back a list of the Citibike stations
# Optionally we pass a query parameter call "name" which we then use to 
# search for Citibike stations that contain that string in their name

@app.route('/citibike_api')
def citibike_stations():


  sql = "SELECT DISTINCT id, name, capacity, lat, lon  FROM status_fall2017"

  search_query = request.args.get('name')
  # If we have a search query, we add a condition in SQL
  if search_query:
    sql += " WHERE name LIKE %(station_name_query)s"


  with engine.connect() as con:
    # If there is a search query, we populate the parameter in SQL
    # Since we want to have an approximate query uisng LIKE, 
    # we put the wildcard character before and after the search string 
    if search_query: 
      params = {"station_name_query": '%'+search_query+'%' }
      stations = pd.read_sql(sql, con=con, params=params)
    else: 
      stations = pd.read_sql(sql, con=con)

  # Convert the returned dataframe into a list of dictionaries
  list_of_stations = stations.to_dict(orient='records')
  api_results = {"stations": list_of_stations}

  # We JSON-ify our dictionary and return it as the API response
  return jsonify(api_results)

In [ ]:
# This API call returns a plot, in JSON format
# Specifically, this returns a 

@app.route('/station_map',  methods=['GET'])
def station_map():

    # Connect to the database, execute the query, and get back the results
    sql = "SELECT DISTINCT id, name, capacity, lat, lon  FROM status_fall2017" 
    with engine.connect() as connection:  
        stations = pd.read_sql(sql, con=connection)

    fig, ax = plt.subplots()
    ax = stations.plot(kind='scatter', x='lon', y='lat', figsize=(6,6), ax=ax)

    buf = BytesIO()
    fig.savefig(buf, format="png")
    # Embed the result in the html output.
    data = base64.b64encode(buf.getbuffer()).decode("ascii")

    # Create the response. We will put the retrieved data as a list of
    # dictionaries, under the key "stations".
    results = {"image": data}

    # We JSON-ify our dictionary and return it as the API response
    return jsonify(results)

In [ ]:
# Start the webserver.

print(f" * ngrok tunnel '{public_url}' -> 'http://127.0.0.1:{port}'")
app.run(use_reloader=False, port=port)